# Import the necessary libraries

In [6]:
import pandas as pd
import sys
import os
import matplotlib.pyplot as plt
import seaborn as sns
import logging
import pandas as pd
import joblib

 
# Configure logging
logging.basicConfig(filename='../logs/',
                    level=logging.INFO,
                    format='%(asctime)s:%(levelname)s:%(message)s')
  

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../../scripts')))
from load_csv_data import Load_CSV_Data


# Credit_Fraud_Detection_Experiment

In [8]:
import mlflow
import mlflow.sklearn

# Set experiment name (same as credit data to log in the same experiment)
mlflow.set_experiment("Credit_Fraud_Detection_Experiment")

# Function to run and log model experiments for fraud data
def log_model_with_mlflow_fraud(model, X_train, X_test, y_train, y_test, model_name):
    with mlflow.start_run(run_name=model_name):
        # Train the model
        model.fit(X_train, y_train)
        
        # Predict
        y_pred = model.predict(X_test)

        # Log parameters and metrics
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        # Log metrics in MLflow
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        # Log the model itself
        mlflow.sklearn.log_model(model, model_name)
        
        print(f"{model_name} logged in MLflow with accuracy: {accuracy}")


# Load data

In [9]:
fraud_df = Load_CSV_Data('../../data/encoded_fraud_data.csv')
fraud_df.load_csv_data()
fraud_df = fraud_df.get_data()

Data successfully loaded from ../../data/encoded_fraud_data.csv


In [10]:
fraud_df.head()

,user_id,purchase_value,age,ip_address,class,lower_bound_ip_address,upper_bound_ip_address,transaction_velocity,purchase_hour,purchase_day_of_week,...,country_Zimbabwe,device_id_target_encoded,signup_year,signup_month,signup_day,signup_hour,purchase_year,purchase_month,purchase_day,time_diff_hours
0,247547,47,30,1.677886e+07,0,16778240.0,16779263.0,3632215.0,3,6,...,False,0.082634,2015,6,28,3,2015,8,9,1008.948611
1,220737,15,34,1.684205e+07,0,16809984.0,16842751.0,1231637.0,20,2,...,False,0.082634,2015,1,28,14,2015,2,11,342.121389
2,390400,44,29,1.684366e+07,0,16843264.0,16843775.0,1997534.0,23,5,...,False,0.081518,2015,3,19,20,2015,4,11,554.870556
3,69592,55,30,1.693873e+07,0,16924672.0,16941055.0,7640897.0,16,5,...,False,0.082634,2015,2,24,6,2015,5,23,2122.471389
4,174987,51,37,1.697198e+07,0,16941056.0,16973823.0,10249579.0,4,1,...,False,0.082634,2015,7,7,12,2015,11,3,2847.105278


# Feature and target separation

In [11]:
# Feature and Target separation for credit card data
X_fraud = fraud_df.drop(columns=['class'])  # Features (remove target column)
y_fraud = fraud_df['class']                # Target (Class column)

# Check the shape of the datasets to ensure separation is correct
print(f"Credit card data - Features: {X_fraud.shape}, Target: {y_fraud.shape}")

# Save X_fraud and y_fraud as CSV files for future use
X_fraud.to_csv('../../data/splittedData/X_fraud.csv', index=False)
y_fraud.to_csv('../../data/splittedData/y_fraud.csv', index=False)

Credit card data - Features: (129146, 205), Target: (129146,)


# Logistic regression

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

# Fraud Data - Logistic Regression
print("\n\nLogistic Regression for Fraud Data")

# Split the fraud data into training and testing sets (80% train, 20% test)
X_train_fraud, X_test_fraud, y_train_fraud, y_test_fraud = train_test_split(X_fraud, y_fraud, test_size=0.2, random_state=42, stratify=y_fraud)

# Initialize the Logistic Regression model
log_reg_fraud = LogisticRegression(max_iter=1000)

log_model_with_mlflow_fraud(log_reg_fraud, X_train_fraud, X_test_fraud, y_train_fraud, y_test_fraud, "Logistic_Regression_Fraud")


# Train the model on the training data
log_reg_fraud.fit(X_train_fraud, y_train_fraud)

# Save the trained Logistic Regression model
joblib.dump(log_reg_fraud, '../saved_models/logistic_regression_fraud_model.pkl')

# Make predictions on the test data
y_pred_fraud = log_reg_fraud.predict(X_test_fraud)

# Evaluate the model performance
accuracy_fraud = accuracy_score(y_test_fraud, y_pred_fraud)
precision_fraud = precision_score(y_test_fraud, y_pred_fraud)
recall_fraud = recall_score(y_test_fraud, y_pred_fraud)
f1_fraud = f1_score(y_test_fraud, y_pred_fraud)

# Print evaluation metrics
print(f"Fraud Data - Accuracy: {accuracy_fraud:.4f}")
print(f"Fraud Data - Precision: {precision_fraud:.4f}")
print(f"Fraud Data - Recall: {recall_fraud:.4f}")
print(f"Fraud Data - F1-Score: {f1_fraud:.4f}")
print("\nClassification Report for Fraud Data:\n", classification_report(y_test_fraud, y_pred_fraud))
print("\nConfusion Matrix for Fraud Data:\n", confusion_matrix(y_test_fraud, y_pred_fraud))



Logistic Regression for Fraud Data


c:\Users\wubeshet.abera\Projects\Ten-Academy\Fraud-detection-system\venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
2024/10/23 08:12:07 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Logistic_Regression_Fraud logged in MLflow with accuracy: 0.9049941927990709
Fraud Data - Accuracy: 0.9050
Fraud Data - Precision: 0.0000
Fraud Data - Recall: 0.0000
Fraud Data - F1-Score: 0.0000

Classification Report for Fraud Data:
               precision    recall  f1-score   support

           0       0.90      1.00      0.95     23376
           1       0.00      0.00      0.00      2454

    accuracy                           0.90     25830
   macro avg       0.45      0.50      0.48     25830
weighted avg       0.82      0.90      0.86     25830


Confusion Matrix for Fraud Data:
 [[23376     0]
 [ 2454     0]]


c:\Users\wubeshet.abera\Projects\Ten-Academy\Fraud-detection-system\venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\wubeshet.abera\Projects\Ten-Academy\Fraud-detection-system\venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\wubeshet.abera\Projects\Ten-Academy\Fraud-detection-system\venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to 

# Decision tree

In [15]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split

# Assuming X_fraud and y_fraud are already defined (features and target)
# Train-test split
X_train_fraud, X_test_fraud, y_train_fraud, y_test_fraud = train_test_split(X_fraud, y_fraud, test_size=0.2, random_state=42)

# Initialize the Decision Tree Classifier
dt_fraud = DecisionTreeClassifier(random_state=42)

log_model_with_mlflow_fraud(dt_fraud, X_train_fraud, X_test_fraud, y_train_fraud, y_test_fraud, "Decision_Tree_Fraud")


# Train the model
dt_fraud.fit(X_train_fraud, y_train_fraud)

# Save the trained  model
joblib.dump(dt_fraud, '../saved_models/decision_tree_fraud_model.pkl')

# Make predictions on the test set
y_pred_fraud = dt_fraud.predict(X_test_fraud)

# Evaluate the model
accuracy_fraud = accuracy_score(y_test_fraud, y_pred_fraud)
precision_fraud = precision_score(y_test_fraud, y_pred_fraud)
recall_fraud = recall_score(y_test_fraud, y_pred_fraud)
f1_fraud = f1_score(y_test_fraud, y_pred_fraud)

# Print the evaluation results
print(f"Fraud Data - Decision Tree Results:")
print(f"Accuracy: {accuracy_fraud:.4f}")
print(f"Precision: {precision_fraud:.4f}")
print(f"Recall: {recall_fraud:.4f}")
print(f"F1-Score: {f1_fraud:.4f}")


2024/10/23 08:20:12 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Decision_Tree_Fraud logged in MLflow with accuracy: 0.989082462253194
Fraud Data - Decision Tree Results:
Accuracy: 0.9891
Precision: 0.9434
Recall: 0.9394
F1-Score: 0.9414


# Random Forest

In [16]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split

# Assuming X_fraud and y_fraud are already defined (features and target)
# Train-test split
X_train_fraud, X_test_fraud, y_train_fraud, y_test_fraud = train_test_split(X_fraud, y_fraud, test_size=0.2, random_state=42)

# Initialize the Random Forest Classifier
rf_fraud = RandomForestClassifier(random_state=42, n_estimators=100)

log_model_with_mlflow_fraud(rf_fraud, X_train_fraud, X_test_fraud, y_train_fraud, y_test_fraud, "Random_Forest_Fraud")


# Train the model
rf_fraud.fit(X_train_fraud, y_train_fraud)

# Save the trained  model
joblib.dump(rf_fraud, '../saved_models/random_forest_fraud_model.pkl')

# Make predictions on the test set
y_pred_fraud = rf_fraud.predict(X_test_fraud)

# Evaluate the model
accuracy_fraud = accuracy_score(y_test_fraud, y_pred_fraud)
precision_fraud = precision_score(y_test_fraud, y_pred_fraud)
recall_fraud = recall_score(y_test_fraud, y_pred_fraud)
f1_fraud = f1_score(y_test_fraud, y_pred_fraud)

# Print the evaluation results
print(f"Fraud Data - Random Forest Results:")
print(f"Accuracy: {accuracy_fraud:.4f}")
print(f"Precision: {precision_fraud:.4f}")
print(f"Recall: {recall_fraud:.4f}")
print(f"F1-Score: {f1_fraud:.4f}")


2024/10/23 08:21:43 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Random_Forest_Fraud logged in MLflow with accuracy: 0.9900890437475803
Fraud Data - Random Forest Results:
Accuracy: 0.9901
Precision: 0.9469
Recall: 0.9469
F1-Score: 0.9469


# Gradient Boosting

In [17]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split

# Assuming X_fraud and y_fraud are already defined (features and target)
# Train-test split
X_train_fraud, X_test_fraud, y_train_fraud, y_test_fraud = train_test_split(X_fraud, y_fraud, test_size=0.2, random_state=42)

# Initialize the Gradient Boosting Classifier
gb_fraud = GradientBoostingClassifier(random_state=42, n_estimators=100, learning_rate=0.1)

log_model_with_mlflow_fraud(gb_fraud, X_train_fraud, X_test_fraud, y_train_fraud, y_test_fraud, "Gradient_Boosting_Fraud")


# Train the model
gb_fraud.fit(X_train_fraud, y_train_fraud)

# Save the trained  model
joblib.dump(gb_fraud, '../saved_models/gradient_boosting_fraud_model.pkl')

# Make predictions on the test set
y_pred_fraud = gb_fraud.predict(X_test_fraud)

# Evaluate the model
accuracy_fraud = accuracy_score(y_test_fraud, y_pred_fraud)
precision_fraud = precision_score(y_test_fraud, y_pred_fraud)
recall_fraud = recall_score(y_test_fraud, y_pred_fraud)
f1_fraud = f1_score(y_test_fraud, y_pred_fraud)

# Print the evaluation results
print(f"Fraud Data - Gradient Boosting Results:")
print(f"Accuracy: {accuracy_fraud:.4f}")
print(f"Precision: {precision_fraud:.4f}")
print(f"Recall: {recall_fraud:.4f}")
print(f"F1-Score: {f1_fraud:.4f}")


2024/10/23 08:23:45 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Gradient_Boosting_Fraud logged in MLflow with accuracy: 0.9905149051490515
Fraud Data - Gradient Boosting Results:
Accuracy: 0.9905
Precision: 0.9733
Recall: 0.9237
F1-Score: 0.9479


# Multi-Layer Perceptron (MLP)

In [18]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split

# Train-test split for fraud data
X_train_fraud, X_test_fraud, y_train_fraud, y_test_fraud = train_test_split(X_fraud, y_fraud, test_size=0.2, random_state=42)

# Initialize the MLP Classifier
mlp_fraud = MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=300, random_state=42)

log_model_with_mlflow_fraud(mlp_fraud, X_train_fraud, X_test_fraud, y_train_fraud, y_test_fraud, "MLP_Fraud")


# Train the model
mlp_fraud.fit(X_train_fraud, y_train_fraud)

# Save the trained  model
joblib.dump(mlp_fraud, '../saved_models/MLP_fraud_model.pkl')

# Make predictions on the test set
y_pred_fraud = mlp_fraud.predict(X_test_fraud)

# Evaluate the model
accuracy_fraud = accuracy_score(y_test_fraud, y_pred_fraud)
precision_fraud = precision_score(y_test_fraud, y_pred_fraud)
recall_fraud = recall_score(y_test_fraud, y_pred_fraud)
f1_fraud = f1_score(y_test_fraud, y_pred_fraud)

# Print the evaluation results
print(f"Fraud Data - MLP Results:")
print(f"Accuracy: {accuracy_fraud:.4f}")
print(f"Precision: {precision_fraud:.4f}")
print(f"Recall: {recall_fraud:.4f}")
print(f"F1-Score: {f1_fraud:.4f}")


2024/10/23 08:26:32 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


MLP_Fraud logged in MLflow with accuracy: 0.9067363530778165
Fraud Data - MLP Results:
Accuracy: 0.9067
Precision: 1.0000
Recall: 0.0008
F1-Score: 0.0017


# Convolutional Neural Network (CNN)

In [19]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import mlflow
import mlflow.keras

# Assuming fraud_df is your DataFrame
# fraud_df = pd.read_csv('your_data.csv')  # Load your dataset here

# Step 1: Check for non-numeric columns and convert them
for column in fraud_df.columns:
    if fraud_df[column].dtype == 'object':
        # Convert object columns to numeric using LabelEncoder
        label_encoder = LabelEncoder()
        fraud_df[column] = label_encoder.fit_transform(fraud_df[column])
    elif fraud_df[column].dtype == 'bool':
        # Convert boolean columns to int (0, 1)
        fraud_df[column] = fraud_df[column].astype(int)

# Step 2: Convert all integer columns to float to ensure compatibility with TensorFlow
fraud_df = fraud_df.astype(float)

# Step 3: Feature and target separation
X_fraud = fraud_df.drop(columns=['class'])  # Assuming 'class' is the target
y_fraud = fraud_df['class']

# Step 4: Reshape the data for CNN (add a channel dimension for 1D CNN)
X_fraud_reshaped = np.expand_dims(X_fraud.values, axis=2)

# Step 5: Train-test split
X_train_fraud, X_test_fraud, y_train_fraud, y_test_fraud = train_test_split(X_fraud_reshaped, y_fraud, test_size=0.2, random_state=42)

# Step 6: CNN model definition
def create_cnn_model(input_shape):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=input_shape))
    model.add(tf.keras.layers.MaxPooling1D(pool_size=2))
    model.add(tf.keras.layers.Conv1D(filters=64, kernel_size=3, activation='relu'))
    model.add(tf.keras.layers.MaxPooling1D(pool_size=2))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(64, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))  # Binary classification
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Step 7: Create and train the CNN model with MLflow logging
with mlflow.start_run(run_name="CNN_Fraud_Detection"):
    cnn_model_fraud = create_cnn_model(input_shape=(X_train_fraud.shape[1], 1))
    
    # Fit the model
    history = cnn_model_fraud.fit(X_train_fraud, y_train_fraud, epochs=10, batch_size=32, validation_split=0.2)
    
    # Log the model and metrics
    mlflow.keras.log_model(cnn_model_fraud, "CNN_Fraud_Model")
    mlflow.log_param("epochs", 10)
    mlflow.log_param("batch_size", 32)
    
    # Evaluate the model
    y_pred_fraud = (cnn_model_fraud.predict(X_test_fraud) > 0.5).astype("int32")
    
    # Calculate and log evaluation metrics
    accuracy_fraud = accuracy_score(y_test_fraud, y_pred_fraud)
    precision_fraud = precision_score(y_test_fraud, y_pred_fraud)
    recall_fraud = recall_score(y_test_fraud, y_pred_fraud)
    f1_fraud = f1_score(y_test_fraud, y_pred_fraud)

    mlflow.log_metric("accuracy", accuracy_fraud)
    mlflow.log_metric("precision", precision_fraud)
    mlflow.log_metric("recall", recall_fraud)
    mlflow.log_metric("f1_score", f1_fraud)

    # Save the trained model using TensorFlow's save method
    cnn_model_fraud.save('../saved_models/CNN_fraud_model.h5')

# Print evaluation results
print(f"Fraud Data - CNN Results:")
print(f"Accuracy: {accuracy_fraud:.4f}")
print(f"Precision: {precision_fraud:.4f}")
print(f"Recall: {recall_fraud:.4f}")
print(f"F1-Score: {f1_fraud:.4f}")


c:\Users\wubeshet.abera\Projects\Ten-Academy\Fraud-detection-system\venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
2583/2583 ━━━━━━━━━━━━━━━━━━━━ 17s 6ms/step - accuracy: 0.8960 - loss: 761049.5625 - val_accuracy: 0.9038 - val_loss: 0.3307
Epoch 2/10
2583/2583 ━━━━━━━━━━━━━━━━━━━━ 17s 6ms/step - accuracy: 0.9040 - loss: 0.3229 - val_accuracy: 0.9038 - val_loss: 0.3166
Epoch 3/10
2583/2583 ━━━━━━━━━━━━━━━━━━━━ 28s 11ms/step - accuracy: 0.9058 - loss: 0.3123 - val_accuracy: 0.9038 - val_loss: 0.3166
Epoch 4/10
2583/2583 ━━━━━━━━━━━━━━━━━━━━ 16s 6ms/step - accuracy: 0.9047 - loss: 0.3147 - val_accuracy: 0.9038 - val_loss: 0.3166
Epoch 5/10
2583/2583 ━━━━━━━━━━━━━━━━━━━━ 12s 5ms/step - accuracy: 0.9049 - loss: 0.3141 - val_accuracy: 0.9038 - val_loss: 0.3166
Epoch 6/10
2583/2583 ━━━━━━━━━━━━━━━━━━━━ 14s 5ms/step - accuracy: 0.9064 - loss: 0.3107 - val_accuracy: 0.9038 - val_loss: 0.3166
Epoch 7/10
2583/2583 ━━━━━━━━━━━━━━━━━━━━ 12s 5ms/step - accuracy: 0.9050 - loss: 0.3139 - val_accuracy: 0.9038 - val_loss: 0.3166
Epoch 8/10
2583/2583 ━━━━━━━━━━━━━━━━━━━━ 12s 5ms/step - accuracy: 0.9039 - l

2024/10/23 08:36:36 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2024/10/23 08:36:43 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


808/808 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step
Fraud Data - CNN Results:
Accuracy: 0.9067
Precision: 0.0000
Recall: 0.0000
F1-Score: 0.0000


c:\Users\wubeshet.abera\Projects\Ten-Academy\Fraud-detection-system\venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# Recurrent Neural Network (RNN)


In [20]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import mlflow
import mlflow.keras
from sklearn.preprocessing import LabelEncoder

# Assuming fraud_df is your DataFrame
# fraud_df = pd.read_csv('your_data.csv')  # Load your dataset here

# Step 1: Check for non-numeric columns and convert them
for column in fraud_df.columns:
    if fraud_df[column].dtype == 'object':
        # Convert object columns to numeric using LabelEncoder
        label_encoder = LabelEncoder()
        fraud_df[column] = label_encoder.fit_transform(fraud_df[column])
    elif fraud_df[column].dtype == 'bool':
        # Convert boolean columns to int (0, 1)
        fraud_df[column] = fraud_df[column].astype(int)

# Step 2: Convert all integer columns to float to ensure compatibility with TensorFlow
fraud_df = fraud_df.astype(float)

# Step 3: Feature and target separation
X_fraud = fraud_df.drop(columns=['class'])  # Assuming 'class' is the target
y_fraud = fraud_df['class']

# Step 4: Train-test split
X_fraud_train, X_fraud_test, y_fraud_train, y_fraud_test = train_test_split(X_fraud, y_fraud, test_size=0.2, random_state=42)

# Step 5: Reshape data to (samples, timesteps, features) for RNN input
X_fraud_train_rnn = np.expand_dims(X_fraud_train.values, axis=1)
X_fraud_test_rnn = np.expand_dims(X_fraud_test.values, axis=1)

# Step 6: Create RNN Model
def create_rnn_model(input_shape):
    model = Sequential()
    
    # RNN layer with 64 units
    model.add(SimpleRNN(units=64, activation='tanh', input_shape=input_shape))
    
    # Dropout to prevent overfitting
    model.add(Dropout(0.5))
    
    # Fully connected layers
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))  # Binary classification output
    
    # Compile the model
    model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

# Step 7: Create and train the RNN model with MLflow logging
with mlflow.start_run(run_name="RNN_Fraud_Detection"):
    rnn_model_fraud = create_rnn_model(X_fraud_train_rnn.shape[1:])
    
    # Fit the model
    history = rnn_model_fraud.fit(X_fraud_train_rnn, y_fraud_train, epochs=10, batch_size=64, validation_split=0.2)
    
    # Log the model and metrics
    mlflow.keras.log_model(rnn_model_fraud, "RNN_Fraud_Model")
    mlflow.log_param("epochs", 10)
    mlflow.log_param("batch_size", 64)
    
    # Evaluate the model
    fraud_pred_rnn = rnn_model_fraud.predict(X_fraud_test_rnn)
    fraud_pred_rnn = np.round(fraud_pred_rnn).astype(int).flatten()

    # Calculate classification report
    report = classification_report(y_fraud_test, fraud_pred_rnn, output_dict=True)

    # Safe extraction of metrics
    accuracy_fraud = report.get('accuracy', 0.0)
    precision_fraud = report.get('1', {}).get('precision', 0.0)
    recall_fraud = report.get('1', {}).get('recall', 0.0)
    f1_fraud = report.get('1', {}).get('f1-score', 0.0)

    # Log metrics
    mlflow.log_metric("accuracy", accuracy_fraud)
    mlflow.log_metric("precision", precision_fraud)
    mlflow.log_metric("recall", recall_fraud)
    mlflow.log_metric("f1_score", f1_fraud)

    # Save the trained model using TensorFlow's save method
    rnn_model_fraud.save('../saved_models/RNN_fraud_model.h5')

# Print evaluation results
print("Fraud Data - RNN Model")
print(classification_report(y_fraud_test, fraud_pred_rnn))


Epoch 1/10


c:\Users\wubeshet.abera\Projects\Ten-Academy\Fraud-detection-system\venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1292/1292 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9046 - loss: 0.3305 - val_accuracy: 0.9038 - val_loss: 0.3247
Epoch 2/10
1292/1292 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9049 - loss: 0.3191 - val_accuracy: 0.9038 - val_loss: 0.3232
Epoch 3/10
1292/1292 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9048 - loss: 0.3172 - val_accuracy: 0.9038 - val_loss: 0.3238
Epoch 4/10
1292/1292 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9042 - loss: 0.3187 - val_accuracy: 0.9038 - val_loss: 0.3221
Epoch 5/10
1292/1292 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9047 - loss: 0.3168 - val_accuracy: 0.9038 - val_loss: 0.3216
Epoch 6/10
1292/1292 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9050 - loss: 0.3160 - val_accuracy: 0.9038 - val_loss: 0.3209
Epoch 7/10
1292/1292 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9044 - loss: 0.3170 - val_accuracy: 0.9038 - val_loss: 0.3171
Epoch 8/10
1292/1292 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9049 - loss: 0.3162 - val_accurac

2024/10/23 08:48:55 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2024/10/23 08:49:02 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


808/808 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
Fraud Data - RNN Model
              precision    recall  f1-score   support

         0.0       0.91      1.00      0.95     23419
         1.0       0.00      0.00      0.00      2411

    accuracy                           0.91     25830
   macro avg       0.45      0.50      0.48     25830
weighted avg       0.82      0.91      0.86     25830



c:\Users\wubeshet.abera\Projects\Ten-Academy\Fraud-detection-system\venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\wubeshet.abera\Projects\Ten-Academy\Fraud-detection-system\venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\wubeshet.abera\Projects\Ten-Academy\Fraud-detection-system\venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` param

# Long Short-Term Memory (LSTM)


In [21]:
import numpy as np
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Assuming X_fraud and y_fraud are already prepared

# Create LSTM Model
def create_lstm_model(input_shape):
    model = Sequential()
    
    # LSTM layer with 64 units
    model.add(LSTM(units=64, activation='tanh', input_shape=input_shape))
    
    # Dropout to prevent overfitting
    model.add(Dropout(0.5))
    
    # Fully connected layers
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))  # Binary classification output
    
    # Compile the model
    model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

# Train-test split for fraud data
X_fraud_train, X_fraud_test, y_fraud_train, y_fraud_test = train_test_split(X_fraud, y_fraud, test_size=0.2, random_state=42)

# Reshape data to (samples, timesteps, features) for LSTM input
X_fraud_train_lstm = np.expand_dims(X_fraud_train, axis=1)
X_fraud_test_lstm = np.expand_dims(X_fraud_test, axis=1)

# Build and train the LSTM model for fraud data
lstm_model_fraud = create_lstm_model(X_fraud_train_lstm.shape[1:])
lstm_model_fraud.fit(X_fraud_train_lstm, y_fraud_train, epochs=10, batch_size=64, validation_split=0.2)

# Evaluate the LSTM model for fraud data
fraud_pred_lstm = lstm_model_fraud.predict(X_fraud_test_lstm)
fraud_pred_lstm = np.round(fraud_pred_lstm)

# Print evaluation results
print("Fraud Data - LSTM Model")
print(classification_report(y_fraud_test, fraud_pred_lstm))

# Save the trained model using TensorFlow's save method
lstm_model_fraud.save('../saved_models/LSTM_fraud_model.h5')


c:\Users\wubeshet.abera\Projects\Ten-Academy\Fraud-detection-system\venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
1292/1292 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9048 - loss: 0.3355 - val_accuracy: 0.9038 - val_loss: 0.3309
Epoch 2/10
1292/1292 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9048 - loss: 0.3167 - val_accuracy: 0.9038 - val_loss: 0.3298
Epoch 3/10
1292/1292 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9047 - loss: 0.3157 - val_accuracy: 0.9038 - val_loss: 0.3320
Epoch 4/10
1292/1292 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9035 - loss: 0.3184 - val_accuracy: 0.9038 - val_loss: 0.3316
Epoch 5/10
1292/1292 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9052 - loss: 0.3144 - val_accuracy: 0.9038 - val_loss: 0.3289
Epoch 6/10
1292/1292 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9042 - loss: 0.3166 - val_accuracy: 0.9038 - val_loss: 0.3249
Epoch 7/10
1292/1292 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9046 - loss: 0.3155 - val_accuracy: 0.9038 - val_loss: 0.3236
Epoch 8/10
1292/1292 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9031 - loss: 0.3190 - 

c:\Users\wubeshet.abera\Projects\Ten-Academy\Fraud-detection-system\venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\wubeshet.abera\Projects\Ten-Academy\Fraud-detection-system\venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\wubeshet.abera\Projects\Ten-Academy\Fraud-detection-system\venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` param

: 